In [1]:
%matplotlib inline
import pandas as pd

In [2]:
compras_df = pd.read_csv('./data/clean_compras.csv')
compras_df.dtypes

NUM BEN           int64
BENEFICIARIO     object
RFC              object
FACTURA          object
CONTRATO         object
CONCEPTO         object
TIPO             object
# OP            float64
FECHA            object
IMPORTE         float64
SHEET            object
dtype: object

## Empezamos con una explansion de datos que tenemos, fecha por ejemplo

In [3]:
pd.to_datetime(compras_df['FECHA'])

0      2020-12-11
1      2020-12-11
2      2020-12-11
3      2020-12-11
4      2020-12-23
          ...    
1535   2020-12-11
1536   2020-12-11
1537   2020-12-16
1538   2020-12-15
1539   2020-12-11
Name: FECHA, Length: 1540, dtype: datetime64[ns]

In [4]:
compras_df['FECHA']=pd.to_datetime(compras_df['FECHA'])

In [5]:
compras_df['DIA'] = compras_df['FECHA'].apply(lambda x: x.day)

In [6]:
compras_df['MES'] = compras_df['FECHA'].apply(lambda x: x.month)


In [7]:
compras_df['ANO'] = compras_df['FECHA'].apply(lambda x: x.year)


Podria haberse hecho de esta forma:

df['DIA'] = df['FECHA'].dt.day

df['MES'] = df['FECHA'].dt.month

df['YEAR'] = df['FECHA'].dt.year

## Proyectemos unas variables contra otras

In [8]:
beneficiarios_sales = compras_df['BENEFICIARIO'].value_counts().to_dict()

In [9]:
list(beneficiarios_sales.keys())[0:5]

['CAMARA MEXICANA DE LA INDUSTRIA DE LA CONSTRUCCION',
 'AUTOKAM REGIOMONTANA,S.A.DE C.V.',
 'CAR ONE MONTERREY, S.A. DE C.V.',
 'PEREZ RIOS RAMIRO',
 'S.I.M.E.P.R.O.D.E.']

In [10]:
compras_df['BENEFICIARIO_SALES'] = compras_df['BENEFICIARIO'].apply(lambda x: beneficiarios_sales[x])

In [11]:
sheet_mean_compras = compras_df.groupby('SHEET')['IMPORTE'].mean()
sheet_mean_compras

SHEET
COMUNICACION              109705.343662
CONTRATISTAS Y FDO FED    474566.765787
GASTOS VARIOS             881659.731797
SERV PPROF                229592.166667
SERV. PERS.                59623.405217
Name: IMPORTE, dtype: float64

In [12]:
sheet_mean_compras.to_dict()

{'COMUNICACION': 109705.34366197183,
 'CONTRATISTAS Y FDO FED': 474566.76578729285,
 'GASTOS VARIOS': 881659.7317971016,
 'SERV PPROF': 229592.16666666666,
 'SERV. PERS.': 59623.40521739131}

In [13]:
compras_df['IMPORTE_GASTO_PROMEDIO'] = compras_df['SHEET'].apply(lambda x: sheet_mean_compras[x])

## Traernos informacion externa, enriquecer los datos usando una tool de paginas de linkedin

In [14]:
pd.DataFrame(compras_df['BENEFICIARIO'].unique()).to_csv('./data/unique_beneficiarios.csv', index=None)

In [15]:
#Cargar
linkedin_mty = pd.read_csv('./data/linkedin_mty.csv')

In [16]:
has_site = {}
for i, ele in linkedin_mty.fillna(True).iterrows():
    has_site[ele['query']] = True if ele['error'] == True else False

In [17]:
has_site

{'Rosa Lorena Valdez Miranda': True,
 'Roque Yañez Ramos': True,
 'Laura Perla Cordova Rodriguez': True,
 'Rosa Ofelia Coronado Flores': True,
 'Alvaro Flores Palomo': True,
 'Liliana Tijerina Cantu': True,
 'Ruben Basaldua Moyeda': True,
 'Maria Cristina Muñoz Rios': True,
 'Gerardo Guerrero Adame': True,
 'Mayela Maria De Lourdes Quiroga Tamez': True,
 'Humberto Arturo Garza De Hoyos': True,
 'Irma Montero Solis': True,
 'Jorge Alejandro Alvarado Martinez': True,
 'Ana Lilia Coronado Araiza': True,
 'Francisco Salazar Guadiana': True,
 'Maria Thalina Almaraz Gonzalez': True,
 'Diego Armando Arellano Aguilar': True,
 'Jorge Adrian Ayala Cantu': True,
 'Adriana Paola Coronado Ramirez': True,
 'LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.': True,
 'GH MAQUINARIA Y EQUIPO, S.A.DE C.V.': True,
 'PLOMIFERRETERA EL TORNILLO, S.A. DE C.V.': False,
 'EQUIPOS Y ACCESORIOS CANTU, S.A. DE C.V.': True,
 'PRAXAIR MEXICO S. DE R.L. DE C.V.': True,
 'COMERCIALIZADORA HNOS.ALVAREZ FLORES,S.A.DE C.V

In [19]:
compras_df['HAS_LINKEDIN_SITE'] = compras_df['BENEFICIARIO'].apply(lambda x: has_site[x.replace('  ', ' ')])
compras_df['HAS_LINKEDIN_SITE']

0       True
1       True
2       True
3       True
4       True
        ... 
1535    True
1536    True
1537    True
1538    True
1539    True
Name: HAS_LINKEDIN_SITE, Length: 1540, dtype: bool

In [20]:
compras_df['HAS_LINKEDIN_SITE'].value_counts()

True     1392
False     148
Name: HAS_LINKEDIN_SITE, dtype: int64

In [22]:
compras_df.to_csv('./data/compras_df_enriched.csv', index=None)